# Step 2: Fill holes in processing area

In [ ]:
import gdal
import numpy as np
import rasterio
from tqdm.notebook import tnrange


ds = gdal.Open('process_area_june_28.tif')
band1 = ds.GetRasterBand(1).ReadAsArray()
band2 = np.copy(band1)

In [ ]:
from tqdm.notebook import tnrange

for iteration in range(0, 2):
    for x_window in tnrange(1, band1.shape[0] - 2, 1):
        for y_window in range(1, band1.shape[1] - 2, 1):
            window = band2[x_window - 1: x_window + 2, y_window - 1: y_window + 2]
            if window[1, 1] == 0:
                u = window[0, 1]
                d = window[2, 1]
                l = window[1, 0]
                r = window[1, 2]
                if (u + d) == 2 or (l + r) == 2:
                    band2[x_window, y_window] = 1.


In [ ]:
for iteration in range(0, 1):
    for x_window in tnrange(2, band1.shape[0] - 3, 1):
        for y_window in range(2, band1.shape[1] - 3, 1):
            window = band1[x_window - 2: x_window + 2, y_window - 2: y_window + 2]
            # -2, -1, 0, 1
            if np.sum(window[1:3, 1:3]) == 0:
                u = np.sum(window[0, 1:3])
                d = np.sum(window[3, 1:3])
                l = np.sum(window[1:3, 0])
                r = np.sum(window[1:3, 3])
                if (u + d) >= 3 or (l + r) >= 3:
                    band2[x_window - 1: x_window +1, y_window -1: y_window + 1] = 1.



In [ ]:
for iteration in range(0, 2):
    for x_window in tnrange(1, band1.shape[0] - 2, 1):
        for y_window in range(1, band1.shape[1] - 2, 1):
            window = band2[x_window - 1: x_window + 2, y_window - 1: y_window + 2]
            if window[1, 1] == 0:
                u = window[0, 1]
                d = window[2, 1]
                l = window[1, 0]
                r = window[1, 2]
                if (u + d) == 2 or (l + r) == 2:
                    band2[x_window, y_window] = 1.

In [ ]:
band1 = np.copy(band2)
for iteration in range(0, 1):
    for x_window in tnrange(2, band1.shape[0] - 3, 1):
        for y_window in range(2, band1.shape[1] - 3, 1):
            window = band1[x_window - 2: x_window + 2, y_window - 2: y_window + 2]
            if np.sum(window[1:3, 1:3]) == 0:
                u = np.sum(window[0, 1:3])
                d = np.sum(window[3, 1:3])
                l = np.sum(window[1:3, 0])
                r = np.sum(window[1:3, 3])
                if (u + d) >= 3 or (l + r) >= 3:
                    band2[x_window - 1: x_window +1, y_window -1: y_window + 1] = 1.

In [ ]:
for iteration in range(0, 2):
    for x_window in tnrange(1, band1.shape[0] - 2, 1):
        for y_window in range(1, band1.shape[1] - 2, 1):
            window = band2[x_window - 1: x_window + 2, y_window - 1: y_window + 2]
            if window[1, 1] == 0:
                u = window[0, 1]
                d = window[2, 1]
                l = window[1, 0]
                r = window[1, 2]
                if (u + d) == 2 or (l + r) == 2:
                    band2[x_window, y_window] = 1.

In [ ]:
transform = rasterio.transform.from_bounds(west = -117.0, south = -55.5,
                                           east = 51.00, north = 37.5000000000,
                                           width = band1.shape[1], 
                                           height = band1.shape[0])

new_dataset = rasterio.open("process_area_june_28_filled_revisit.tif", 'w', driver = 'GTiff',
                           height = band2.shape[0], width = band2.shape[1], count = 1,
                           dtype = 'uint16',#str(stacked.dtype),
                           crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs',
                           transform=transform)
new_dataset.write(band2, 1)
new_dataset.close()

In [ ]:
np.sum(band2)

# Step 1: Convert to 6 x 6 km binary raster

In [ ]:
import gdal
import numpy as np
ds = gdal.Open('sa_africa_binary_new.tif')

options_str = '-ot Byte'

ds = gdal.Translate('new_coords.tif', ds, projWin = [-117.0, 37.5, 51.00, -55.5],
                   options=options_str)
ds = None

In [ ]:
ds = gdal.Open('lulc_sa_africa_binary.tif')
band1 = ds.GetRasterBand(1).ReadAsArray()

In [ ]:
band1 = band1[:33480, :60480]

band1 = np.reshape(band1, ((33480 // 20, 20, 60480 // 20, 20)))
band1 = np.sum(band1, axis = (1, 3))
band1 = band1.astype(np.uint16)

In [ ]:
print(f"The total TOF area is: {np.sum(band1) * ((300*300) / (1000*1000))}")



In [ ]:
import rasterio
transform = rasterio.transform.from_bounds(west = -117.0, south = -55.5,
                                           east = 51.00, north = 37.5000000000,
                                           width = band1.shape[1], 
                                           height = band1.shape[0])

new_dataset = rasterio.open("output_summed.tif", 'w', driver = 'GTiff',
                           height = band1.shape[0], width = band1.shape[1], count = 1,
                           dtype = 'uint16',#str(stacked.dtype),
                           crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs',
                           transform=transform)
new_dataset.write(band1, 1)
new_dataset.close()

In [ ]:
import gdal
import numpy as np
ds = gdal.Open('output_summed.tif')
band1 = ds.GetRasterBand(1).ReadAsArray()

In [ ]:
print(f"The total TOF area is: {np.sum(band1) * ((300*300) / (1000*1000))}")



In [ ]:
for thresh in [0.2, 0.25, 0.3, 0.35]:
    test = np.copy(band1)
    test[test < thresh * 400] = 0
    test[test >= thresh * 400] = 1
    print(np.sum(test) * 36)# / np.sum(band1))# / 9161269)
    #print(test[:100, :100])
    #print(test)

In [ ]:
test = np.copy(band1)
test[test < 0.35 * 400] = 0.
test[test >= 0.35 * 400] = 1.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20,17))
plt.imshow(test, cmap='Greens', vmin=0, vmax=1)
plt.show()

In [ ]:
import rasterio
transform = rasterio.transform.from_bounds(west = -117.0, south = -55.5,
                                           east = 51.00, north = 37.5000000000,
                                           width = band1.shape[1], 
                                           height = band1.shape[0])

new_dataset = rasterio.open("processing_area_mar_17.tif", 'w', driver = 'GTiff',
                           height = test.shape[0], width = test.shape[1], count = 1,
                           dtype = 'uint16',#str(stacked.dtype),
                           crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs',
                           transform=transform)
new_dataset.write(test, 1)
new_dataset.close()

# Step 3: Add in all urban areas

The threshold for urban areas is 20/400 = 5%
The overall thres4hold is 35%

In [ ]:
import gdal
import numpy as np
ds = gdal.Open('sa_africa_lulc.tif')
band1 = ds.GetRasterBand(1).ReadAsArray()

In [ ]:
band1[band1 == 10] = 255.
band1[band1 == 20] = 255.
band1[band1 < 255 ] = 0.
band1[band1 == 255] = 1.
print(np.sum(band1))

In [ ]:
print(band1.shape)

In [ ]:
band1 = band1[:33480, :60480]

band1 = np.reshape(band1, ((33480 // 20, 20, 60480 // 20, 20)))

In [ ]:
band1 = np.sum(band1, axis = (1, 3))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20,17))
plt.imshow(band1, cmap='Greens', vmin=0, vmax=1)
plt.show()

In [ ]:
band1[band1 < 40] = 0.
band1[band1 >= 40] = 1.

In [ ]:
band1 = band1.astype(np.uint16)

In [ ]:
import rasterio
transform = rasterio.transform.from_bounds(west = -117.0, south = -55.5,
                                           east = 51.00, north = 37.5000000000,
                                           width = band1.shape[1], 
                                           height = band1.shape[0])

new_dataset = rasterio.open("crop_6km.tif", 'w', driver = 'GTiff',
                           height = band1.shape[0], width = band1.shape[1], count = 1,
                           dtype = 'uint16',#str(stacked.dtype),
                           crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs',
                           transform=transform)
new_dataset.write(band1, 1)
new_dataset.close()

In [ ]:
np.sum(band1)

In [ ]:
band1.shape

# Step X add in mosaic cropland

In [ ]:
import numpy as np
import rasterio

ds = rasterio.open('sa_africa_lulc.tif')
band1 = ds.read(1)

band1[band1 == 30] = 255.
band1[band1 == 40] = 255.
band1[band1 < 255 ] = 0.
band1[band1 == 255] = 1.
print(np.sum(band1))

band1 = band1[:33480, :60480]

band1 = np.reshape(band1, ((33480 // 20, 20, 60480 // 20, 20)))
band1 = np.sum(band1, axis = (1, 3))

band1[band1 < 80] = 0.
band1[band1 >= 80] = 1.
band1 = band1.astype(np.uint16)

transform = rasterio.transform.from_bounds(west = -117.0, south = -55.5,
                                           east = 51.00, north = 37.5000000000,
                                           width = band1.shape[1], 
                                           height = band1.shape[0])

new_dataset = rasterio.open("crop_mosaic_6km.tif", 'w', driver = 'GTiff',
                           height = band1.shape[0], width = band1.shape[1], count = 1,
                           dtype = 'uint16',#str(stacked.dtype),
                           crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs',
                           transform=transform)
new_dataset.write(band1, 1)
new_dataset.close()

# Step X.2 Add in mosaic herbaceous

In [ ]:
import numpy as np
import rasterio

ds = rasterio.open('tifs/sa_africa_lulc.tif')
band1 = ds.read(1)

band1[band1 == 110] = 255.
band1[band1 < 255 ] = 0.
band1[band1 == 255] = 1.
print(np.sum(band1))

band1 = band1[:33480, :60480]

band1 = np.reshape(band1, ((33480 // 20, 20, 60480 // 20, 20)))
band1 = np.sum(band1, axis = (1, 3))

band1[band1 < 80] = 0.
band1[band1 >= 80] = 1.
band1 = band1.astype(np.uint16)

transform = rasterio.transform.from_bounds(west = -117.0, south = -55.5,
                                           east = 51.00, north = 37.5000000000,
                                           width = band1.shape[1], 
                                           height = band1.shape[0])

new_dataset = rasterio.open("tifs/herb_mosaic_6km.tif", 'w', driver = 'GTiff',
                           height = band1.shape[0], width = band1.shape[1], count = 1,
                           dtype = 'uint16',#str(stacked.dtype),
                           crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs',
                           transform=transform)
new_dataset.write(band1, 1)
new_dataset.close()

# Step 4: Combine the tifs

In [ ]:
ds = gdal.Open('tof_processing_area_june_23.tif')
process_area = ds.GetRasterBand(1).ReadAsArray()
ds = None

ds = gdal.Open('urban_6km.tif')
urban = ds.GetRasterBand(1).ReadAsArray()
ds = None

ds = gdal.Open('crop_6km.tif')
crop = ds.GetRasterBand(1).ReadAsArray()
ds = None

In [ ]:
process_area = process_area + urban + crop
process_area[process_area > 1] = 1.

In [ ]:
import rasterio
transform = rasterio.transform.from_bounds(west = -117.0, south = -55.5,
                                           east = 51.00, north = 37.5000000000,
                                           width = band1.shape[1], 
                                           height = band1.shape[0])

new_dataset = rasterio.open("process_area_june_28.tif", 'w', driver = 'GTiff',
                           height = band1.shape[0], width = band1.shape[1], count = 1,
                           dtype = 'uint16',#str(stacked.dtype),
                           crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs',
                           transform=transform)
new_dataset.write(process_area, 1)
new_dataset.close()

# Step 5: Compare the coverage

In [ ]:
import gdal
import numpy as np
import rasterio
from tqdm.notebook import tnrange

In [ ]:
ds = gdal.Open('sa_africa_lulc.tif')
lulc = ds.GetRasterBand(1).ReadAsArray()
lulc = lulc[:, :60480]
ds = None

ds = gdal.Open('process_area_june_28_filled.tif')
area = ds.GetRasterBand(1).ReadAsArray()
area = area.repeat(20,axis=0).repeat(20,axis=1)
ds = None

In [ ]:
unique_all, counts_all = np.unique(lulc, return_counts=True)
for x, y in zip(unique_all, counts_all):
    print(f"{x}: {y}")

In [ ]:
for x, y, z in zip(unique_process[1:], counts_process[1:], counts_all):
    if x in unique_all:
    print(f"{x}: {y}, {y/z}")

In [ ]:
lulc[area == 0] = 0.

unique_process, counts_process = np.unique(lulc, return_counts=True)

In [ ]:
import pandas as pd
process = pd.DataFrame({"id": unique_process, "count": counts_process})
total =  pd.DataFrame({"id": unique_all, "count_all": counts_all})

In [ ]:
overall = pd.merge(process, total)

In [ ]:
overall['percent'] = overall['count'] / overall['count_all']

In [ ]:
names = ['Crop rainfed', 'Herbaceous', 'Tree or shrub', 'Crop irrigated',
        'Mosaic cropland' ,'Mosaic natural', '']

In [ ]:
overall.to_csv("coverage.csv")

# More workspace

In [ ]:
import numpy as np
import rasterio

ds = rasterio.open('tifs/process_area_june_28_filled.tif')
current_area = ds.read(1)

In [ ]:
np.sum(current_area)

In [ ]:
ds = rasterio.open('tifs/herb_mosaic_6km.tif')
crop_area = ds.read(1)
np.sum(np.logical_or(crop_area == 1, current_area == 1))